# Exploring DyNetX
DyNetx - Dynamic Network library

http://dynetx.readthedocs.io/en/latest/index.html

In [1]:
import numpy as np
import networkx as nx
import dynetx as dn

Note: needed to update Numpy to 1.14.x

In [2]:
np.version.full_version

'1.14.2'

Create a **dynamic graph**, and add a few vertices.

In [60]:
# A DynGraph stores nodes and timestamped interaction.
G = dn.DynGraph()

In [61]:
G.add_node(1)

In [62]:
G.add_nodes_from([2,3,4])

In [63]:
G.number_of_nodes()

4

In [64]:
G.nodes()

[1, 2, 3, 4]

We add **interactions** to a dynamic graph. An interaction is an edge between two vertices, and they are added at a specific time (here the timestamp is `t=0`)

In [65]:
time = 0
G.add_interactions_from([(1, 2), (1, 3), (2, 3), (3, 4)], t=time)

In [67]:
time = 1
G.add_interactions_from([(1, 4)], t=1)
G.

and we can see the interactions, and for a specific time

In [38]:
G.interactions()

[(1, 2, {'t': [[0, 0]]}),
 (1, 3, {'t': [[0, 0]]}),
 (1, 4, {'t': [[1, 1]]}),
 (2, 3, {'t': [[0, 0]]}),
 (3, 4, {'t': [[0, 0]]})]

In [39]:
G.interactions(t=1)

[(1, 4, {'t': [1]})]

In [40]:
for i in G.stream_interactions():
    print(i)

(1, 2, '+', 0)
(1, 3, '+', 0)
(2, 3, '+', 0)
(3, 4, '+', 0)
(1, 4, '+', 1)


how to build own own representation

In [80]:
# Specify the node-to-node interactions over a span of time
# (u, v) : [(t, e), (t, e)]
# The nodes are: u, v
# The time span is 
#   t: appearance snapshot id
#.  e: vanishing snapshot id
form1 = {
    (1, 2): [(0, 1)],
    (1, 3): [(0, 9)],
    (1, 4): [(1, 3), (6, 7)],
    (2, 3): [(0, 1), (3, 7)],
    (2, 4): [],
    (3, 4): [(0, 9)]
}

In [73]:
G = dn.DynGraph()

In [75]:
for link in form1:
    u = link[0]
    v = link[1]
    for interaction in form1[link]:
        t = interaction[0]
        e = interaction[1]
        G.add_interaction(u, v, t, e)

In [76]:
G.interactions()

[(1, 2, {'t': [[0, 0]]}),
 (1, 3, {'t': [[0, 8]]}),
 (1, 4, {'t': [[1, 2], [6, 6]]}),
 (2, 3, {'t': [[0, 0], [3, 6]]}),
 (3, 4, {'t': [[0, 8]]})]

In [77]:
for i in G.stream_interactions():
    print(i)

(1, 2, '+', 0)
(1, 3, '+', 0)
(2, 3, '+', 0)
(3, 4, '+', 0)
(1, 2, '-', 1)
(1, 4, '+', 1)
(2, 3, '-', 1)
(1, 4, '-', 3)
(2, 3, '+', 3)
(1, 4, '+', 6)
(1, 4, '-', 7)
(2, 3, '-', 7)
(1, 3, '-', 9)
(3, 4, '-', 9)
